In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
from pathlib import Path

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

sys.path.append("/home/acho/Sync/Python/sigtools")
# from sigtools.representations import *
# from sigtools.sounds import *
# from sigtools.processing import *
# from sigtools.spatialization import *

# Difference between RMS and mean of abs

In [ ]:
x1 = np.linspace(0, 100*2*pi, 10000) + 2*np.pi*np.random.rand()
# x2 = x1 - np.pi
x2 = 100*x1 + 2*np.pi*np.random.rand()

y1 = 90*np.sin(x1)
y2 = 90*np.sin(x2)
y3 = y1 - y2

print(RMS(y3) - np.mean(np.abs(y3)))

plt.plot(np.abs(y3))
plt.show()

In [ ]:
_ = plt.hist(np.abs(y3), bins=100)

# Trajectory simulations

In [ ]:
from scipy.interpolate import interp1d

T = 10
rate1 = 2
rate2 = 2

# 90 degrees out of phase
angle1 = 180*(np.random.rand() - 0.5)
dir1_R = np.random.choice([True, False])
if angle1 >= 0 and dir1_R: #RR
    angle2 = angle1 - 90
    dir2_R = dir1_R
elif angle1 < 0 and dir1_R: #RL
    angle2 = -(angle1 + 90)
    dir2_R = not dir1_R
elif angle1 < 0 and not dir1_R: #LR
    angle2 = angle1 + 90
    dir2_R = dir1_R
else: #LR
    angle2 = 90 - angle1
    dir2_R = not dir1_R

# angle1 = 13.3
# # angle1 = 90*np.random.choice([-1, 1])
# # angle1 = 180*(np.random.rand() - 0.5)
# angle2 = angle1 + 90
# # angle2 = -angle1
# # angle2 = 180*(np.random.rand() - 0.5)
# # dir1_R = True
# dir1_R = np.random.choice([True, False])
# dir2_R = dir1_R
# # dir2_R = np.random.choice([True, False])

traj1 = make_circular_sinuisoidal_trajectory(2, T, 100, 0, rate1, angle1, dir1_R)
traj2 = make_circular_sinuisoidal_trajectory(2, T, 100, 0, rate2, angle2, dir2_R)
theta1 = traj_to_theta(traj1)
theta2 = traj_to_theta(traj2)
N1 = len(theta1)
N2 = len(theta2)

if N2 > N1:
    x = np.arange(N1)
    y = theta1
    f = interp1d(x, y)
    x_new = np.linspace(0, x[-1], N2)
    theta1 = f(x_new)
else:
    x = np.arange(N2)
    y = theta2
    f = interp1d(x, y)
    x_new = np.linspace(0, x[-1], N1)
    theta2 = f(x_new)

# print(RMS(theta1))
# print(RMS(theta2))
print(angle1, angle2)
print(dir1_R, dir2_R)
print(np.mean(np.abs(theta1 - theta2)))
print(RMS(theta1 - theta2))

plt.plot(theta1[0:400])
plt.plot(theta2[0:400])
# plt.plot(theta1 - theta2)
plt.show()

# Generate figures to validate synthesis/also check RMS value after moving

In [ ]:
def woodworth(theta):
    rads = 2*np.pi*theta/360
    a = 87.5e-3
    c = 343
    return_vals = np.arange(len(rads))
    for i, rad in enumerate(rads):
        if rad >= 0 and rad <= np.pi/2:
            return_vals[i] = a/c*(rad + np.sin(rad))
        elif rad >= np.pi/2 and rad <= np.pi:
            return_vals[i] =  a/c*(np.pi - rad + np.sin(rad))
        elif rad <= 0 and rad >= -np.pi/2:
            rad = np.abs(rad)
            return_vals[i] = -a/c*(rad + np.sin(rad))
        elif rad <= -np.pi/2 and rad >= -np.pi:
            return_vals[i] = -a/c*(np.pi - rads + np.sin(rad))
        else:
            raise ValueError("invalid theta")
    return return_vals

In [ ]:
T = 1
fs = 44100
gn1 = GaussianNoise(T, 44100).make_binaural()
gn2 = GaussianNoise(T, 44100).make_binaural()

r = 100
rate1 = 1
rate2 = 10
init_angle1 = 0
init_angle2 = 0
init_R = True
traj1 = make_circular_sinuisoidal_trajectory(2, T, r, 0, rate1, init_angle1, init_R)
traj2 = make_circular_sinuisoidal_trajectory(2, T, r, 0, rate2, init_angle2, init_R)
thetas1 = traj_to_theta(traj1)
thetas2 = traj_to_theta(traj2)

moved_gn1 = move_sound(traj1, gn1)
moved_gn2 = move_sound(traj2, gn2)

In [ ]:
print(RMS(moved_gn1.data))
print(RMS(moved_gn1.data[:, 0]))
print(RMS(moved_gn1.data[:, 1]))
plt.plot(moved_gn1.data)
plt.show()

In [ ]:
print(RMS(moved_gn2.data))
print(RMS(moved_gn2.data[:, 0]))
print(RMS(moved_gn2.data[:, 1]))
plt.plot(moved_gn2.data)
plt.show()

In [ ]:
print(RMS( (moved_gn1 + moved_gn2).data ))
print(RMS( (moved_gn1.data + moved_gn2.data)[:, 0] ))
print(RMS( (moved_gn1.data + moved_gn2.data)[:, 1] ))
plt.plot(moved_gn1.data + moved_gn2.data)
plt.show()

In [ ]:
iac_dur = 5e-3 # Analysis window = 5 ms
iac = InterauralCues(moved_gn2, iac_dur)
ILDs = iac.ILDs
ITDs = iac.ITDs
ILDs[-1] = 0
ITDs[np.where(np.abs(ITDs) > 1e-3)[0]] = 0

# ILDs = ILDs[:-1]
# ITDs = 1e6*ITDs[:-1]
ITDs = 1e6*ITDs

a = 87.5e-3
c = 343
rads2 = 2*np.pi*thetas2/360
woodworth2 = 1e6*a/c*(rads2 + np.sin(rads2))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 4), constrained_layout=True)
axes[0].plot(thetas2, "k-", alpha=0.8)
axes[1].plot(ILDs, "k-", alpha=0.8)
axes[2].plot(ITDs, "k-", alpha=0.8)
# axes[2].plot(np.linspace(0, len(ITDs), len(woodworth2)), woodworth2, "k--", alpha=0.5)

axes[0].set_xlim((0, len(thetas2)))
axes[1].set_xlim((0, len(ILDs)))
axes[2].set_xlim((0, len(ITDs)))

axes[0].set_xticks([round(i*len(thetas2)/5) for i in range(6)])
axes[1].set_xticks([round(i*len(ITDs)/5) for i in range(6)])
axes[2].set_xticks([round(i*len(ILDs)/5) for i in range(6)])

axes[0].set_xticklabels([])
axes[1].set_xticklabels([])
axes[2].set_xticklabels(["{:.1f}".format(i) for i in np.arange(0, T + 0.01, 0.2)], fontsize=10)

axes[2].set_xlabel("Time [s]", fontsize=14)

axes[0].set_yticks(np.arange(-90, 91, 45))
axes[1].set_yticks(np.arange(-20, 21, 10))
axes[2].set_yticks(np.arange(-800, 801, 400))

axes[0].set_yticklabels(np.arange(-90, 91, 45), fontsize=10)
axes[1].set_yticklabels(np.arange(-20, 21, 10), fontsize=10)
axes[2].set_yticklabels(np.arange(-800, 801, 400), fontsize=10)

axes[0].set_ylim((-90, 90))
axes[1].set_ylim((-22, 22))
axes[2].set_ylim((-800, 800))

axes[0].set_ylabel(r"$\theta$", fontsize=14)
axes[1].set_ylabel("ILD [dB]", fontsize=14)
axes[2].set_ylabel("ITD [$\mu$s]", fontsize=14)

axes[0].grid(linestyle=":")
axes[1].grid(linestyle=":")
axes[2].grid(linestyle=":")

# axes[0].set_title(r"\textbf{Trajectory validation}", fontsize=20)

plt.show()
# fig.savefig("validate_synthesis.pdf")